# Mini Service mit Python

Diese Übung baut auf der "Das Garagentor" (IoT MQTT HTTP) Übung auf. 

In der Übung das Garangentor bauen wir einen kleinen Backend Service. Bevor ein Service deployed wird, ist es essenziell Bausteine für den Betrieb und Wartung einzubauen. 
Zwei zentrale Aspekte schauen wir uns an: 

  - Konfigurationsparameter (z.B. MQTT Host, Username, usw.) gehören in eine separate Konfigurationsdatei
  - Protokollierung: Nachvollziehbarkeit (insbesondere von Exceptions) gewährleisten

Damit ein Service stabil funktioniert, gehören noch weitere, vor allem im Bezug auf die Code Struktur, dazu. 
Weiter gehört auch das Sammeln von Metriken dazu, um zum Beispiel bei hoher Auslastung des Services entsprechende Massnahmen ergreifen zu können.   

## Lerninhalte
  - Konfigurationsdateien mit [YAML](https://yaml.org/)
  - Protokollierung - [Python logging library](https://docs.python.org/3/library/logging.html)



# Protokollieren

Eine Logging Library ist Teil jeder erfolgreichen Programmiersprache. Entweder gleich Integriert (wie bei Python) oder als separate Library (Java [Log4j](https://logging.apache.org/log4j/2.x/)). 

Eines der wichtigen Konzepte bei der Protokollierung ist das Logging Level. Dazu gibt es gute Artikel im WWW: 
 - [Python Logging Levels Explained](https://www.logicmonitor.com/blog/python-logging-levels-explained)
 - [Real Python - Logging](https://realpython.com/python-logging/)

**Challenge 1:** Passe das nachfolgende Skript so an, dass die Meldung in Zeile 5 nicht auf der Konsole ausgegeben wird, aber die Meldung in Zeile 6 schon. 

**Hinweis:** `force=True` in der Zeile 3 ist nur gesetzt, weil sonst der Jupyter Kernel für jede Ausführung neu gestartet werden müsste. Manual Hinweis zu Force: `If this keyword  is specified as true, any existing handlers attached to the root logger are removed and closed, before carrying out the configuration as specified by the other arguments.` (Beschreibung Aufrufen: In Pycharm mit gedrückter `Ctrl` Taste auf `basicConfig` klicken).  
  

In [13]:
import logging
import os.path
import sys

logging.basicConfig(level=logging.WARNING, force=True)

logging.debug('This will not get logged')
logging.info('This will get logged')

### Log-Dateien

Läuft ein Service im Hintergrund und möchte der Administrator z.B. einen Fehler Diagnostizieren, sind Log Dateien äusserst praktisch. 
Nicht immer müssen Anwendungen selbst Log-Dateien erstellen, denn in gewissen Anwendungsfällen reicht es, wenn der Service auf das Terminal protokolliert, weil ein übergeordneter Service den `Standard Output` abfangt und in einer übergeordnete Anwendung protokolliert (z.B. bei Container: [Docker Logging: A Complete Guide](https://sematext.com/guides/docker-logs/)). Vor der Implementation der Protokollierung empfiehlt es sich immer zu prüfen, wie die Anwendung oder der Service in Zukunft ausgeführt wird. Denn dieses Kriterium ist für die Umsetzung ausschlaggebend. 

Unser Service soll zukünftig direkt via systemctl in einer Debian Umgebung ausgeführt werden. Um die Protokollierung möchten wir uns selber kümmern. 

**Challenge 2:** Passe das nachfolgende Skript so an, dass die Meldung in Zeile 4 protokolliert wird und dass die Log-Datei nicht jedes Mal zu Begin gelöscht wird, sondern die Einträge angefügt werden. Tipp: Alle Änderungen erfolgen in der Zeile 3. 

In [14]:
import logging

logging.basicConfig(filename='app.log', filemode='a', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', force=True, level=logging.WARNING)
logging.warning('This will get logged to a file')



In **Challenge 2** haben wir erfolgreich in eine Datei protokolliert. Nun kommen noch zwei weitere Anforderungen hinzu:
  - Die Log-Dateien sollen nach ein paar Tagen gelöscht werden (Log-Rotation)
  - Die Ausgabe soll gleichzeitig auf dem Terminal und in die Datei erfolgen.

Dafür definieren wir mehrere Handlers. Ein `StreamHandler` (Protokollierung in die Konsole) wird standardmässig hinzugefügt und in `basicConfig` konfiguriert. Wenn in `basicConfig` die Option `filename` gesetzt wird, wird stattdessen ein Handler konfiguriert, der in eine Datei protokolliert. In nachfolgenden Code wurde ein zusätzlicher Handler definiert, welcher nicht nur in eine Datei protokolliert, sondern die Log-Datei täglich versioniert. Log-Dateien die älter als 10 Tage sind (`backupCount`) werden automatisch gelöscht. 

**Challenge 3:** Passe den nachfolgenden Code so an, dass die Aussagen in Zeile 16 und 17 stimmen, ohne dabei die Zeilen 16 und 17 zu ändern. 


In [21]:
import logging.handlers

LOG_FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
#Chnage logging type
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT, force=True)

logger = logging.getLogger()

fh = logging.handlers.TimedRotatingFileHandler(filename='mylog.log', when="midnight", backupCount=10)
fh.setFormatter(logging.Formatter(LOG_FORMAT))

fh.setLevel(logging.ERROR)
logger.addHandler(fh)

logger.debug("We only need this line when we are debugging and not in our log file.")
logger.info("We want this line in our log file and in our Console.")

2024-01-24 10:54:09,418 - root - INFO - We want this line in our log file and in our Console.


### Uncaught Exception

Wenn Exceptions auftreten, die nicht explizit abgefangen und geloggt werden, dann werden diese nur auf dem `Standard Output` ausgegeben. Mit einer kleinen Erweiterung können wir dem Python Interpreter sagen, dass er alle Exceptions protokollieren soll. 

Das nachfolgende Skript funktioniert innerhalb von Jupyter nicht, das Jupyter selbst die Exception bereits abfängt und im Output darstellt. Um das nachfolgende Skript auszuprobieren, muss es in einer separaten Python Skript Datei ausgeführt werden. 

In [30]:
import logging.handlers
import sys

LOG_FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

logging.basicConfig(level=logging.ERROR, format=LOG_FORMAT, force=True)

logger = logging.getLogger()

def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))

sys.excepthook = handle_exception

# Invalid conversion
a = int("Hallo")


ValueError: invalid literal for int() with base 10: 'Hallo'

# Konfigurationsdateien mit YAML

Es gibt verschiedene Arten von Konfigurationsdateiformaten. Eine Möglichkeit ist der Einsatz von [YAML](https://yaml.org/). 

Für diese YAML Beispiele und Übungen müssen wir zuerst ein paar Dinge vorbereiten: 
 - `pyyaml` in `requirements.txt` aufnehmen und installieren.  
 - Datei `example.yaml` mit nachfolgendem Inhalt anlegen:
```yaml
event-name: "Flywheel show"
guests:
  - "John Doe"
  - "Jane Smith"
  - "Alex Johnson"
  - "Emily Davis"
  - "Michael Brown"
```

Weitere Informationen und Beispiele: [Python YAML](https://python.land/data-processing/python-yaml)

**Challenge 4:** Erweitere den nachfolgenden Code so, dass die ersten drei Gäste in einer Zeile (Komma getrennt) ausgegeben werden. 

**Hinweis:** Die `pyyaml` Library lädt die YAML-Datei und wandelt sie in entsprechende Python Typen um. Es hilft sich den Inhalt von `event_config` mit dem Debugger anzuschauen. 

In [27]:
import yaml

with open('example.yaml', "r") as file:
    event_config = yaml.safe_load(file)

first_three_guests = event_config["guests"][:3]
first_three_guests_str = ", ".join(first_three_guests)

print(f'Our brand new {event_config["event-name"]} has a total of {len(event_config["guests"])} guests.')
print(f'The first three guests are: {first_three_guests_str}.')


Our brand new Flywheel show has a total of 5 guests.
The first three guests are: John Doe, Jane Smith, Alex Johnson.


Startet ein Service zum ersten Mal, dann hat der User möglicherweise die Konfigurationsdatei noch nicht angelegt. Um die Benutzerfreundlichkeit eines Services zu erhöhen, können wir eine Beispielkonfiguration beim ersten Start gleich anlegen. 

**Challenge 5:** Erweitere den nachfolgenden Code so, dass folgende Beispielkonfigurationsdatei erstellt wird: 
```yaml
event-name: "REPLACE WITH EVENT NAME"
guests:
  - "ADD GUEST NAMES HERE"
```

**Wichtig:** Es ist in diesem Fall keine schöne Lösung einfach die Beispielkonfigurationsdatei in einen String zu packen und in eine Datei zu schreiben. 

In [29]:
import os
import sys

CONFIG_FILENAME = "config.yaml"

if not os.path.exists(CONFIG_FILENAME):
    example_config = {"event-name": "REPLACE WITH EVENT NAME"}
    with open(CONFIG_FILENAME, 'w') as fp:
        yaml.safe_dump(example_config, fp)
    logging.error(f"Configuration file missing. Wrote example configuration file {CONFIG_FILENAME}. Aborting start. ")
    sys.exit(1)



# Code-Struktur

Nachdem wir die ersten Erfahrungen mti *Logging* und *YAML* gesammelten haben, wollen wir unseren Service, welchen wir im **Challenge 15** in der vorherigen Übung erstellt haben, mit den neuen "Features" erweitern.  

Anstatt die Logging-Konfiguration und die YAML-Konfiguration einfach oben im Garagentor-Skript einzufügen, möchten wir unseren Code ein wenig strukturieren. 

Dafür erstellen wir zwei weitere Dateien bzw. Python Module `app_logger.py` und `app_config.py`. Insgesamt haben wir dann drei Module:
 - `app_logger.py` - Initialisierung der Protokollierung
 - `app_config.py` - Laden und initialisieren der Konfigurationsdatei
 - `main.py` - Der Hauptteil unserer Anwendung, der die beiden Module `app_logger` und `app_config` importiert. 

### Challenge 7: Protokollierung
Ergänze deinen Garagentor-Überwachungsservice mit einer Protokollierung 
Ziele: 
 - Logging in Datei `app_logger.py` definiert und initialisiert
 - `main.py` importiert `app_logger`
 - Protokolliert wird Level *DEBUG* auf das Terminal und Level *INFO* in eine Log-Datei.  
 - Protokolliert wird mit entsprechendem Log Level: 
   - INFO: Wenn das Garagentor geöffnet wird.
   - INFO: Wenn das Garagentor geschlossen wird. 
   - INFO: Wenn das Garagentor zu lange offen steht. 
   - INFO: Wenn sich der MQTT Client erfolgreich verbunden hat.
   - DEBUG: Wenn der MQTT Client eine Message empfangt, Topic und Message protokollieren
 - `print` wird weder in `main.py` noch in `app_logger.py` verwendet. 
 - *Uncaught exceptions* werden abgefangen und protokolliert 

### Challenge 8: Konfigurationsdatei
Ergänze deinen Garagentor-Überwachungsservice mit einer Konfigurationsdatei
 - Konfigurationsdatei wird in Datei `app_config.py` geladen und als Variable `config` zur Verfügung gestellt. 
 - Wenn keine Konfigurationsdatei vorhanden ist, wird diese automatisch erstellt, die Anwendung bricht den Start in diesem Fall ab und gibt eine Meldung aus. 
 - Die Konfigurationsdatei beinhaltet:
   - GARAGE_DOOR_TOPIC
   - PUSHOVER_API_TOKEN 
   - PUSHOVER_USER_ID 
   - Meldungstext, der an den Mitbewohner geschickt wird: 'Schliess das verdammte Garagentor...'
   - MQTT_HOST, MQTT_USER
 - In `main.py` wird mit `app_config.config` auf die Konfiguration zurückgegriffen. 
 - Die Key-Strings sind in `app_config.py` definiert.
 - In `main.py` wird auf die Werte in der Konfigurationsdatei zurückgegriffen. (D.h. es steht nicht mehr `"cloud.tbz.ch"` in `client.connect(`, sondern es wird der Wert aus der Konfigurationsdatei verwendet. 

Beispiel verwendung von `app_config` in `main.py`: 
```python
import app_config

logging.info(f"Connecting to {app_config.config[app_config.MQTT_HOST]}")
...
```  

Beispiel von Key-Strings in `app_config.py`:
```python
...
MQTT_HOST = "mqtt-host"
MQTT_PORT = "mqtt-port"
...
```
